# <span style="color:magenta"> BEDtools test. </span>

This notebook combines the TE model data with the gene model data, as they were previously separated.

1. Get the annotations file (.gff3) and the reference file (ctg.fa)
2. SeqIO.parse through reference genome and get a list of all contig names.
3. Make pandas dataframe of annotation file.
4. Subset dataframe to column 0 that only has values in list of contig names from reference file.
5. Use "to.csv(\t)" to convert the subset to a tab-separated GFF file.
6. Cat this subset GFF file of TE data to the gene data GFF file.

In [8]:
import pybedtools
from pybedtools import BedTool
import os
import glob
import pprint
import numpy # needed for last few bedtools functions
import scipy

In [9]:
#First we need to define the base dirs
DIRS ={}
DIRS['BASE'] = '/home/anjuni/methylation_calling/pacbio'
DIRS['BED_INPUT'] = os.path.join(DIRS['BASE'], 'input', 'sorted_bed_files')
DIRS['GFF_INPUT'] = os.path.join('/home/anjuni/analysis/output')
DIRS['BED_OUT'] = os.path.join(DIRS['BASE'], 'output', 'intersected_bed_files')

In [10]:
#Quick chech if directories exist
for value in DIRS.values():
    if not os.path.exists(value):
        print('%s does not exist' % value)
    else:
        print(value)

/home/anjuni/methylation_calling/pacbio
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files
/home/anjuni/analysis/output does not exist
/home/anjuni/methylation_calling/pacbio/output/intersected_bed_files


In [11]:
#Make filepaths
bed_file_list = [fn for fn in glob.iglob('%s/*.bed' % DIRS['BED_INPUT'], recursive=True)]
ph_Pst_104E_gff = os.path.join(DIRS['GFF_INPUT'], 'Pst_104E_v13_ph_ctg_combined_sorted_anno.gff3')

In [12]:
# not sure how to use it as it doesn't have absolute path
all_bed_files = os.listdir(DIRS['BED_INPUT'])
print(all_bed_files)

['5mC_hc_nanopolish_sorted.bed', 'filtered_bed', '5mC_tombo_sorted.bed', '6mA_tombo_sorted.bed', '6mA_smrtlink_sorted.bed', '6mA_prob_smrtlink_sorted.bed']


In [13]:
#Check that the list works
print(*bed_file_list, sep='\n')

/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_hc_nanopolish_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_tombo_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_tombo_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_smrtlink_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_prob_smrtlink_sorted.bed


In [14]:
# Testing out making a BedT
# Setting BedTools objects
m5c_hc_nano = BedTool(bed_file_list[0])
m5c_tombo = BedTool(bed_file_list[1])
m5c_nano = BedTool(bed_file_list[2])
m6a_tombo = BedTool(bed_file_list[3])
m6a_smrt = BedTool(bed_file_list[4])
m6a_prob_smrt = BedTool(bed_file_list[5])
anno_ph_gff = BedTool(ph_Pst_104E_gff)

IndexError: list index out of range

In [15]:
# Using a dictionary to make a list of bed files
BED = {}
for file in bed_file_list:
    name = (file[63:-4])
    bed_file = BedTool(file)
    BED[name] = bed_file

In [16]:
pprint.pprint(BED) # see if it works

{'5mC_hc_nanopolish_sorted': <BedTool(/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_hc_nanopolish_sorted.bed)>,
 '5mC_tombo_sorted': <BedTool(/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_tombo_sorted.bed)>,
 '6mA_prob_smrtlink_sorted': <BedTool(/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_prob_smrtlink_sorted.bed)>,
 '6mA_smrtlink_sorted': <BedTool(/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_smrtlink_sorted.bed)>,
 '6mA_tombo_sorted': <BedTool(/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_tombo_sorted.bed)>}


In [17]:
# Make output file handles
hc_nano_tombo_m5c = DIRS['BED_OUT'] = os.path.join(DIRS['BASE'], 'output', 'intersected_bed_files', '5mC_hc_nano_tombo.bed')
nano_tombo_m5c = DIRS['BED_OUT'] = os.path.join(DIRS['BASE'], 'output', 'intersected_bed_files', '5mC_nano_tombo.bed')
tombo_hc_nano_m5c = DIRS['BED_OUT'] = os.path.join(DIRS['BASE'], 'output', 'intersected_bed_files', '5mC_tombo_hc_nano.bed')
tombo_nano_5mc = DIRS['BED_OUT'] = os.path.join(DIRS['BASE'], 'output', 'intersected_bed_files', '5mC_tombo_nano.bed')

#for trying to filter the tombo files
hc_tombo_m5c = DIRS['BED_OUT'] = os.path.join(DIRS['BASE'], 'output', 'intersected_bed_files', '5mC_hc_tombo_sorted.bed')
hc_tombo_m6a = DIRS['BED_OUT'] = os.path.join(DIRS['BASE'], 'output', 'intersected_bed_files', '6mA_hc_tombo_sorted.bed')

# <span style="color:crimson"> Intersections. </span>

In [24]:
# Intersect the high confidence nanopolish 5mC methylation sites with the tombo ones
m5c_hc_nano.intersect(m5c_tombo).saveas(hc_nano_tombo_m5c)

<BedTool(/home/anjuni/methylation_calling/pacbio/output/intersected_bed_files/5mC_hc_nano_tombo.bed)>

In [27]:
# Intersect the low confidence nanopolish 5mC methylation sites with the tombo ones
m5c_nano.intersect(m5c_tombo).saveas(nano_tombo_m5c)

<BedTool(/home/anjuni/methylation_calling/pacbio/output/intersected_bed_files/5mC_nano_tombo.bed)>

In [166]:
m5c_tombo.intersect(m5c_nano).saveas(tombo_nano_5mc)

<BedTool(/home/anjuni/methylation_calling/pacbio/output/intersected_bed_files/5mC_tombo_nano.bed)>

In [9]:
%prun m5c_tombo.intersect(m5c_hc_nano).saveas(tombo_hc_nano_m5c)

In [22]:
m5c_hc_nano.head()

hcontig_000_003	289	290	5mC	0.071	+
 hcontig_000_003	438	439	5mC	0.048	+
 hcontig_000_003	443	444	5mC	0.048	+
 hcontig_000_003	823	824	5mC	0.091	+
 hcontig_000_003	851	852	5mC	0.071	+
 hcontig_000_003	931	932	5mC	0.071	+
 hcontig_000_003	935	936	5mC	0.071	+
 hcontig_000_003	1004	1005	5mC	0.071	+
 hcontig_000_003	1074	1075	5mC	0.071	+
 hcontig_000_003	1077	1078	5mC	0.071	+
 

In [257]:
genes = gff.intersect(tombo_hc_nano)

In [259]:
genes.saveas(os.path.join(DIRS['BASE'], 'output', 'intersected_bed_files', 'gff_tombo_5mC.bed'))

<BedTool(/home/anjuni/methylation_calling/pacbio/output/intersected_bed_files/gff_tombo_5mC.bed)>

In [255]:
tombo_hc_nano = BedTool(tombo_hc_nano_m5c)

In [256]:
gff = BedTool(ph_Pst_104E_gff)

# <span style="color:deeppink"> Intervals. </span>

In [10]:
feature = m5c_hc_nano[0]

In [13]:
print(feature)

hcontig_000_003	289	290	5mC	0.071	+



In [90]:
features = m5c_hc_nano[1:3] # slice does not seem to work

In [28]:
print(features) # bedtool object supports slices but it doesn't show

In [46]:
features = m5c_hc_nano[0:4] # need to name features before each loop because the loop consumes the iterator
for line in features:       # need a loop to view slice
    print(line.score)
    print(len(line))        # difference between start and stop

0.071
1
0.048
1
0.048
1
0.091
1


In [23]:
feature.chrom # only for one line

'hcontig_000_003'

In [48]:
feature.fields[3] # indexed column

'5mC'

In [33]:
len(feature)

1

In [58]:
# viewing gff file attributes with bedtools
interval = anno_ph_gff[1]
sorted(interval.attrs.items())

[('ID', 'evm.model.hcontig_000_003.1'),
 ('Parent', 'evm.TU.hcontig_000_003.1'),
 ('locus_tag', 'Pst104E_15928')]

# <span style="color:mediumvioletred"> Filtering. </span>

In [ ]:
#input files were produced above

#output files for trying to filter the tombo files
hc_tombo_m5c = DIRS['BED_OUT'] = os.path.join(DIRS['BASE'], 'output', 'intersected_bed_files', '5mC_hc_tombo_sorted.bed')
hc_tombo_m6a = DIRS['BED_OUT'] = os.path.join(DIRS['BASE'], 'output', 'intersected_bed_files', '6mA_hc_tombo_sorted.bed')

In [18]:
# define function to filter
def score_filter(feature, L):
    "Returns True if feature is longer than L"
    return float(feature.score) > L

In [22]:
# filter out scores that are zero
filtered_tombo_m5c = BED['5mC_tombo_sorted'].filter(score_filter, 0)
filtered_tombo_m6a = BED['6mA_tombo_sorted'].filter(score_filter, 0)

In [20]:
# try filtering again with the original (manually generated) bed files
filtered_tombo_m5c = m5c_tombo.filter(score_filter, 0)

In [23]:
filtered_tombo_m5c.saveas(hc_tombo_m5c)

<BedTool(/home/anjuni/methylation_calling/pacbio/output/intersected_bed_files/5mC_hc_tombo_sorted.bed)>

In [24]:
filtered_tombo_m6a.saveas(hc_tombo_m6a)

<BedTool(/home/anjuni/methylation_calling/pacbio/output/intersected_bed_files/6mA_hc_tombo_sorted.bed)>

In [145]:
type(m5c_tombo)

pybedtools.bedtool.BedTool

In [107]:
type(m5c_hc_nano)

pybedtools.bedtool.BedTool

# <span style="color:orchid"> Each. </span>

In [193]:
# applies a function to each line
# use output = to save to a file as well
# use u=true to to something else
# try these out on example_file a and b

In [238]:
print(a, '\n', b)

chr1	1	100	feature1	0	+
chr1	100	200	feature2	0	+
chr1	150	500	feature3	0	-
chr1	900	950	feature4	0	+
 
 chr1	155	200	feature5	0	-
chr1	800	901	feature6	0	+



In [239]:
ab = a.intersect(b, u=True) # -u means show the full feature of A, not just the overlapping region

In [241]:
print(ab)

chr1	100	200	feature2	0	+
chr1	150	500	feature3	0	-
chr1	900	950	feature4	0	+



In [242]:
ab = a.intersect(b) # by default, only the overlapping region is shown

In [243]:
print(ab)

chr1	155	200	feature2	0	+
chr1	155	200	feature3	0	-
chr1	900	901	feature4	0	+



In [244]:
ab = a.intersect(b, c=True) # -c ads count column

In [245]:
print(ab)

chr1	1	100	feature1	0	+	0
chr1	100	200	feature2	0	+	1
chr1	150	500	feature3	0	-	1
chr1	900	950	feature4	0	+	1



In [246]:
ab = a.intersect(b, v=True) # like grep -v, -v gives non-overlapping features

In [247]:
print(ab)

chr1	1	100	feature1	0	+



In [248]:
ab = a.intersect(b, s=True) # only matching strand features given

In [249]:
print(ab)

chr1	155	200	feature3	0	-
chr1	900	901	feature4	0	+



In [250]:
ab = a.intersect(b, S=True) # only opposite strand features given

In [251]:
print(ab)

chr1	155	200	feature2	0	+



# <span style="color:fuchsia"> Randomisation. </span>

In [ ]:
# assigns a significance value to the overlap between two bed files
# randomly shuffle a file many times, each time intersecting with another file and counting intersections.
# overlap of original file is compared to empirical distribution of overlaps of shuffled files to get p-value

In [186]:
chromsizes = {'chr1': (0, 1000)} # first, need to set chromosome sizes, so use reference genome fasta for this!
a = pybedtools.example_bedtool('a.bed').set_chromsizes(chromsizes) # set the chromosome sizes for the first bed file
b = pybedtools.example_bedtool('b.bed')

In [187]:
a.head()

chr1	1	100	feature1	0	+
 chr1	100	200	feature2	0	+
 chr1	150	500	feature3	0	-
 chr1	900	950	feature4	0	+
 

In [185]:
b.head()

chr1	155	200	feature5	0	-
 chr1	800	901	feature6	0	+
 

In [183]:
len(a)

4

In [184]:
len(b)

2

In [188]:
results = a.randomintersection(b, iterations=100, shuffle_kwargs={'chrom': True}, debug=True) # set debug=False normally

In [189]:
results = list(results) # normally do 1000 or 10000 randomisations
len(results)

100

In [190]:
print(results[:10])

[0, 3, 1, 1, 2, 2, 2, 1, 4, 2]


In [192]:
results_dict = a.randomstats(b, iterations=100, shuffle_kwargs={'chrom': True}, debug=True)

In [195]:
pprint.pprint(results_dict) # percentile of actual within the distribution percentiles gives the p-value, normalised is the score

{'/home/anjuni/anaconda3/lib/python3.6/site-packages/pybedtools/test/data/a.bed': 4,
 '/home/anjuni/anaconda3/lib/python3.6/site-packages/pybedtools/test/data/b.bed': 2,
 'actual': 3,
 'file_a': '/home/anjuni/anaconda3/lib/python3.6/site-packages/pybedtools/test/data/a.bed',
 'file_b': '/home/anjuni/anaconda3/lib/python3.6/site-packages/pybedtools/test/data/b.bed',
 'frac randomized above actual': 0.01,
 'frac randomized below actual': 0.87,
 'iterations': 100,
 'lower_2.5th': 0.0,
 'median randomized': 2.0,
 'normalized': 1.5,
 'other': 2,
 'percentile': 93.5,
 'self': 4,
 'upper_97.5th': 3.0}


In [196]:
keys = ['self', 'other', 'actual', 'median randomized', 'normalized', 'percentile']
for key in keys:
    print('%s: %s' % (key, results_dict[key]))

self: 4
other: 2
actual: 3
median randomized: 2.0
normalized: 1.5
percentile: 93.5


In practice, a comparison between two sets of features (say, two transcription factors) with 1000 randomizations will have an empirical p-value of < 0.001. That is, out of all the randomizations performed, every single one had fewer intersections than the original. Of course the resolution of the p-value is dependent on the number of randomizations: the lowest nonzero p-value for 10000 iterations will be 0.0001. Getting a non-zero p-value often requires doing more randomizations than is practical (several million to tens of millions).

That's where the enrichment score comes in. The randomized intersections typically have a normal distribution, but just in case, we take the median of the randomized intersections and call this the background or control. Then we divide the actual intersections by this median to get an enrichment score.

The advantage to using the enrichment score is that it gives nonzero scores for more fine-grained comparison among sets of features without performing impractical amounts of randomization. The first example of its usage that I'm aware of is Negre et al. (2010) PLoS Genet 6(1): e1000814, The downside of this metric is that the numbers are relative, and have their greatest utility for making biological conclusions when used in large matrices of pairwise comparisons.

BedTool.randomintersection() and BedTool.randomstats() both use the intersection count method. That is, for each randomization the calculated metric is "number of intersection events". An alternative is to compute the Jaccard statistic on each iteration, as implemented in BedTool.naive_jaccard(). The Jaccard statistic (or Jaccard similarity) is the ratio of the intersection over the union, and is introduced in a genomic intersection context in Favorov et al. (2012) PLoS Comput Biol 8(5): e1002529. However, this still has the same p-value resolution limitation, so the actual-divided-by-median approach could be tried here as well.

# <span style="color:orange"> History. </span>

In [199]:
print(a.history) # use the tags given to find an instance of a bedtool, and the functions to recreate it

[]


In [200]:
a = pybedtools.example_bedtool('a.bed')
>>> b = pybedtools.example_bedtool('b.bed')
>>> c = a.intersect(b)
>>> d = c.slop(g=pybedtools.chromsizes('hg19'), b=1)
>>> e = d.merge()

>>> # this step adds complexity!
>>> f = e.subtract(b)

In [205]:
h = f.history

In [217]:
print(h) # can't really see the steps separated

[[[[<HistoryStep> BedTool("/home/anjuni/anaconda3/lib/python3.6/site-packages/pybedtools/test/data/a.bed").intersect("/home/anjuni/anaconda3/lib/python3.6/site-packages/pybedtools/test/data/b.bed", ), parent tag: olljamny, result tag: ildowzuw], <HistoryStep> BedTool("/tmp/pybedtools.50k6v2sp.tmp").slop(g=OrderedDict([('chr1', (0, 249250621)), ('chr2', (0, 243199373)), ('chr3', (0, 198022430)), ('chr4', (0, 191154276)), ('chr5', (0, 180915260)), ('chr6', (0, 171115067)), ('chr7', (0, 159138663)), ('chr8', (0, 146364022)), ('chr9', (0, 141213431)), ('chr10', (0, 135534747)), ('chr11', (0, 135006516)), ('chr12', (0, 133851895)), ('chr13', (0, 115169878)), ('chr14', (0, 107349540)), ('chr15', (0, 102531392)), ('chr16', (0, 90354753)), ('chr17', (0, 81195210)), ('chr18', (0, 78077248)), ('chr19', (0, 59128983)), ('chr20', (0, 63025520)), ('chr21', (0, 48129895)), ('chr22', (0, 51304566)), ('chrX', (0, 155270560)), ('chrY', (0, 59373566)), ('chrM', (0, 16571)), ('chr6_ssto_hap7', (0, 492856

# <span style="color:teal"> Piping. </span>

In [229]:
>>> x1 = a.intersect(b, u=True)
>>> x2 = x1.merge()

>>> x3 = a.intersect(b, u=True).merge()

>>> x2 == x3

True

In [230]:
x4 = a.intersect(b, u=True).saveas('a-with-b.bed').merge().saveas('a-with-b-merged.bed')

In [236]:
>>> x4 = a\
... .intersect(b, u=True)\
... .saveas('a-with-b.bed')\
... .merge()\
... .saveas('a-with-b-merged.bed')

In [218]:
>>> x5 = a.intersect(b, u=True) # u only sets the actual parts of a that overlapped with a part of b
>>> x6 = a + b                  # this part isn't relevant to methylation since its by each base
                                # but it might be useful when comparing patterns of methylation...
>>> x5 == x6

True

In [222]:
x0 = a.intersect(b)
x5 == x0

False

In [223]:
x0.head()

chr1	155	200	feature2	0	+
 chr1	155	200	feature3	0	-
 chr1	900	901	feature4	0	+
 

In [221]:
x5.head()

chr1	100	200	feature2	0	+
 chr1	150	500	feature3	0	-
 chr1	900	950	feature4	0	+
 

In [224]:
>>> x7 = a.intersect(b, v=True) # v gives the full feature of a
>>> x8 = a - b

>>> x7 == x8

True

In [225]:
x10 = a.intersect(b)

In [227]:
x10.head()

chr1	155	200	feature2	0	+
 chr1	155	200	feature3	0	-
 chr1	900	901	feature4	0	+
 

In [237]:
x9 = (a + b).merge()
x2 == x3 == x4 == x9

True

In [ ]:
# have a look at a.window_maker and a.window